## State Population and Vaccination Data

In [1]:
# Import Pandas, Config Variables, and SQLalchemy
import pandas as pd
from config import endpoint, username, password
from sqlalchemy import create_engine

In [2]:
# Read CSVs for State Data
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")
state_vacc_df = pd.read_csv("../data/state_covid_vaccinations_cleaned.csv")

In [3]:
# State Populations
state_pop_df.head()

,state,population
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [4]:
# State Vaccinations
state_vacc_df.head()

,state,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alaska,154325,71907,21096,9829,59449,8126,12352,1688
1,Alabama,493125,165919,10057,3384,144429,2946,21345,435
2,Arkansas,361550,173312,11981,5743,144792,4798,26541,879
3,Arizona,797550,303899,10957,4175,262120,3601,41613,572
4,California,4379500,1633875,11084,4135,1335886,3381,293834,744


In [5]:
# State Data Merged
state_merged_df = pd.merge(state_pop_df, state_vacc_df, how="left", on="state")
state_merged_df.head()

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
0,Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435
1,Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688
2,Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572
3,Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879
4,California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744


In [6]:
# Set State as the Index
state_info_df = state_merged_df.set_index("state") 
state_info_df.head()

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K
state,,,,,,,,,
Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435
Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688
Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572
Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879
California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744


In [7]:
# Add Empty Column for New Calculation
state_info_df["percent_vaccinated"] = ""
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,percent_vaccinated
state,,,,,,,,,,
Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435,
Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688,
Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572,
Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879,
California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744,
Colorado,5758736,657250,353194,11413,6133,296816,5154,54474,946,
Connecticut,3565287,435075,264707,12203,7425,230913,6477,28602,802,
Delaware,973764,106600,49369,10947,5070,41900,4303,7355,755,
District of Columbia,705749,75175,55108,10652,7808,43377,6146,11685,1656,


In [8]:
# Calcuation for percent vaccinated by state
for index, row in state_info_df.iterrows():
    population = row["population"]
    vaccinated = row["total_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_info_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [9]:
# Check the data types
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   population                        51 non-null     int64 
 1   total_distributed                 51 non-null     int64 
 2   total_administered                51 non-null     int64 
 3   distributed_per_100K              51 non-null     int64 
 4   administered_per_100K             51 non-null     int64 
 5   people_with_1plus_doses           51 non-null     int64 
 6   people_with_1plus_doses_per_100K  51 non-null     int64 
 7   people_with_2_doses               51 non-null     int64 
 8   people_with_2_doses_per_100K      51 non-null     int64 
 9   percent_vaccinated                51 non-null     object
dtypes: int64(9), object(1)
memory usage: 6.9+ KB


In [10]:
# Convert percent vaccinated into a float
state_info_df = state_info_df.astype({"percent_vaccinated": 'float64'})
state_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   population                        51 non-null     int64  
 1   total_distributed                 51 non-null     int64  
 2   total_administered                51 non-null     int64  
 3   distributed_per_100K              51 non-null     int64  
 4   administered_per_100K             51 non-null     int64  
 5   people_with_1plus_doses           51 non-null     int64  
 6   people_with_1plus_doses_per_100K  51 non-null     int64  
 7   people_with_2_doses               51 non-null     int64  
 8   people_with_2_doses_per_100K      51 non-null     int64  
 9   percent_vaccinated                51 non-null     float64
dtypes: float64(1), int64(9)
memory usage: 6.9+ KB


In [11]:
state_info_df.head(10)

,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,percent_vaccinated
state,,,,,,,,,,
Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435,3.38
Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688,9.83
Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572,4.18
Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879,5.74
California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744,4.14
Colorado,5758736,657250,353194,11413,6133,296816,5154,54474,946,6.13
Connecticut,3565287,435075,264707,12203,7425,230913,6477,28602,802,7.42
Delaware,973764,106600,49369,10947,5070,41900,4303,7355,755,5.07
District of Columbia,705749,75175,55108,10652,7808,43377,6146,11685,1656,7.81


## NIH AWS Data / NYT

### Set up State Master DF

In [12]:
# Read NYT / NIH json 
nytimes_nih_covid_df = pd.read_json("../data/daily_covid_stats_by_state_nyt.json", lines=True)

In [13]:
# Get Data for Most Recent Provided Date
most_recent_date = "2021-01-28"
latest_nyt_stateData = nytimes_nih_covid_df[nytimes_nih_covid_df['date'] == f'{most_recent_date}']

In [14]:
latest_nyt_stateData.head(10)

,date,state,fips,cases,deaths
18219,2021-01-28,Alabama,1,452734,7340
18220,2021-01-28,Alaska,2,53524,253
18221,2021-01-28,Arizona,4,745976,12861
18222,2021-01-28,Arkansas,5,290856,4784
18223,2021-01-28,California,6,3272349,39527
18224,2021-01-28,Colorado,8,394668,5670
18225,2021-01-28,Connecticut,9,248765,7020
18226,2021-01-28,Delaware,10,76495,1075
18227,2021-01-28,District of Columbia,11,36132,902
18228,2021-01-28,Florida,12,1687586,26034


In [15]:
# Merge with State Info DF that contains population and vaccinations
latest_data_merged = pd.merge(state_info_df, latest_nyt_stateData, on="state")
latest_data_merged.head(10)

,state,population,total_distributed,total_administered,distributed_per_100K,administered_per_100K,people_with_1plus_doses,people_with_1plus_doses_per_100K,people_with_2_doses,people_with_2_doses_per_100K,percent_vaccinated,date,fips,cases,deaths
0,Alabama,4903185,493125,165919,10057,3384,144429,2946,21345,435,3.38,2021-01-28,1,452734,7340
1,Alaska,731545,154325,71907,21096,9829,59449,8126,12352,1688,9.83,2021-01-28,2,53524,253
2,Arizona,7278717,797550,303899,10957,4175,262120,3601,41613,572,4.18,2021-01-28,4,745976,12861
3,Arkansas,3017804,361550,173312,11981,5743,144792,4798,26541,879,5.74,2021-01-28,5,290856,4784
4,California,39512223,4379500,1633875,11084,4135,1335886,3381,293834,744,4.14,2021-01-28,6,3272349,39527
5,Colorado,5758736,657250,353194,11413,6133,296816,5154,54474,946,6.13,2021-01-28,8,394668,5670
6,Connecticut,3565287,435075,264707,12203,7425,230913,6477,28602,802,7.42,2021-01-28,9,248765,7020
7,Delaware,973764,106600,49369,10947,5070,41900,4303,7355,755,5.07,2021-01-28,10,76495,1075
8,District of Columbia,705749,75175,55108,10652,7808,43377,6146,11685,1656,7.81,2021-01-28,11,36132,902
9,Florida,21477737,2546050,1239975,11854,5773,1125338,5240,111763,520,5.77,2021-01-28,12,1687586,26034


In [16]:
# Create a Master Dataframe to start working with
state_master_df = latest_data_merged[['state', 'date', 'population', 'cases', 'deaths', 'total_distributed', 'total_administered', 'percent_vaccinated' ]]
state_master_df.head(10)

,state,date,population,cases,deaths,total_distributed,total_administered,percent_vaccinated
0,Alabama,2021-01-28,4903185,452734,7340,493125,165919,3.38
1,Alaska,2021-01-28,731545,53524,253,154325,71907,9.83
2,Arizona,2021-01-28,7278717,745976,12861,797550,303899,4.18
3,Arkansas,2021-01-28,3017804,290856,4784,361550,173312,5.74
4,California,2021-01-28,39512223,3272349,39527,4379500,1633875,4.14
5,Colorado,2021-01-28,5758736,394668,5670,657250,353194,6.13
6,Connecticut,2021-01-28,3565287,248765,7020,435075,264707,7.42
7,Delaware,2021-01-28,973764,76495,1075,106600,49369,5.07
8,District of Columbia,2021-01-28,705749,36132,902,75175,55108,7.81
9,Florida,2021-01-28,21477737,1687586,26034,2546050,1239975,5.77


In [17]:
# Create empty column to calculate estimated percent infected to date
state_master_df["est_percent_infected_to_date"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,date,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,2021-01-28,4903185,452734,7340,493125,165919,3.38,
1,Alaska,2021-01-28,731545,53524,253,154325,71907,9.83,
2,Arizona,2021-01-28,7278717,745976,12861,797550,303899,4.18,
3,Arkansas,2021-01-28,3017804,290856,4784,361550,173312,5.74,
4,California,2021-01-28,39512223,3272349,39527,4379500,1633875,4.14,
5,Colorado,2021-01-28,5758736,394668,5670,657250,353194,6.13,
6,Connecticut,2021-01-28,3565287,248765,7020,435075,264707,7.42,
7,Delaware,2021-01-28,973764,76495,1075,106600,49369,5.07,
8,District of Columbia,2021-01-28,705749,36132,902,75175,55108,7.81,
9,Florida,2021-01-28,21477737,1687586,26034,2546050,1239975,5.77,


In [18]:
# Calcuation for est percent infected by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    state_master_df.at[index, "est_percent_infected_to_date"] = percent_infected

In [19]:
state_master_df.head(10)

,state,date,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date
0,Alabama,2021-01-28,4903185,452734,7340,493125,165919,3.38,9.25
1,Alaska,2021-01-28,731545,53524,253,154325,71907,9.83,7.32
2,Arizona,2021-01-28,7278717,745976,12861,797550,303899,4.18,10.27
3,Arkansas,2021-01-28,3017804,290856,4784,361550,173312,5.74,9.65
4,California,2021-01-28,39512223,3272349,39527,4379500,1633875,4.14,8.29
5,Colorado,2021-01-28,5758736,394668,5670,657250,353194,6.13,6.86
6,Connecticut,2021-01-28,3565287,248765,7020,435075,264707,7.42,6.99
7,Delaware,2021-01-28,973764,76495,1075,106600,49369,5.07,7.86
8,District of Columbia,2021-01-28,705749,36132,902,75175,55108,7.81,5.13
9,Florida,2021-01-28,21477737,1687586,26034,2546050,1239975,5.77,7.87


In [20]:
# Add Estimated Percent Immune Column
state_master_df["est_percent_immune"] = ""
state_master_df.head(10)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,state,date,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,2021-01-28,4903185,452734,7340,493125,165919,3.38,9.25,
1,Alaska,2021-01-28,731545,53524,253,154325,71907,9.83,7.32,
2,Arizona,2021-01-28,7278717,745976,12861,797550,303899,4.18,10.27,
3,Arkansas,2021-01-28,3017804,290856,4784,361550,173312,5.74,9.65,
4,California,2021-01-28,39512223,3272349,39527,4379500,1633875,4.14,8.29,
5,Colorado,2021-01-28,5758736,394668,5670,657250,353194,6.13,6.86,
6,Connecticut,2021-01-28,3565287,248765,7020,435075,264707,7.42,6.99,
7,Delaware,2021-01-28,973764,76495,1075,106600,49369,5.07,7.86,
8,District of Columbia,2021-01-28,705749,36132,902,75175,55108,7.81,5.13,
9,Florida,2021-01-28,21477737,1687586,26034,2546050,1239975,5.77,7.87,


In [21]:
# Calcuation for estimated immune by state
for index, row in state_master_df.iterrows():
    population = row["population"]
    infected = row["cases"]
    deaths = row["deaths"]
    vaccinated = row["total_administered"]
    
    est_total_immune = infected + vaccinated
    est_population = population - deaths
    
    percent_immune = round((est_total_immune / est_population) * 100, 2)
    
    state_master_df.at[index, "est_percent_immune"] = percent_immune

In [22]:
state_master_df.head(10)

,state,date,population,cases,deaths,total_distributed,total_administered,percent_vaccinated,est_percent_infected_to_date,est_percent_immune
0,Alabama,2021-01-28,4903185,452734,7340,493125,165919,3.38,9.25,12.64
1,Alaska,2021-01-28,731545,53524,253,154325,71907,9.83,7.32,17.15
2,Arizona,2021-01-28,7278717,745976,12861,797550,303899,4.18,10.27,14.45
3,Arkansas,2021-01-28,3017804,290856,4784,361550,173312,5.74,9.65,15.41
4,California,2021-01-28,39512223,3272349,39527,4379500,1633875,4.14,8.29,12.43
5,Colorado,2021-01-28,5758736,394668,5670,657250,353194,6.13,6.86,13
6,Connecticut,2021-01-28,3565287,248765,7020,435075,264707,7.42,6.99,14.43
7,Delaware,2021-01-28,973764,76495,1075,106600,49369,5.07,7.86,12.94
8,District of Columbia,2021-01-28,705749,36132,902,75175,55108,7.81,5.13,12.94
9,Florida,2021-01-28,21477737,1687586,26034,2546050,1239975,5.77,7.87,13.65


### Set up Daily Cases DF

In [23]:
nyt_avg_daily_cases = nytimes_nih_covid_df
nyt_avg_daily_cases.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [24]:
nyt_avg_daily_cases = nyt_avg_daily_cases.groupby("date").sum()
nyt_avg_daily_cases.head()

,fips,cases,deaths
date,,,
2020-01-21,53,1,0
2020-01-22,53,1,0
2020-01-23,53,1,0
2020-01-24,70,2,0
2020-01-25,76,3,0


In [25]:
nyt_avg_daily_cases["daily_new_cases"] = ""

In [26]:
nyt_avg_daily_cases.head()

,fips,cases,deaths,daily_new_cases
date,,,,
2020-01-21,53,1,0,
2020-01-22,53,1,0,
2020-01-23,53,1,0,
2020-01-24,70,2,0,
2020-01-25,76,3,0,


In [27]:
previous_day = 0

In [28]:
for index, row in nyt_avg_daily_cases.iterrows():
    new_cases = row["cases"] - previous_day
    previous_day = row["cases"]
    nyt_avg_daily_cases.at[index, "daily_new_cases"] = new_cases

In [29]:
nyt_avg_daily_cases["date"] = ""

In [30]:
nyt_avg_daily_cases

,fips,cases,deaths,daily_new_cases,date
date,,,,,
2020-01-21,53,1,0,1,
2020-01-22,53,1,0,0,
2020-01-23,53,1,0,0,
2020-01-24,70,2,0,1,
2020-01-25,76,3,0,1,
...,...,...,...,...,...
2021-01-24,1762,25177522,419207,129527,
2021-01-25,1762,25333204,421114,155682,
2021-01-26,1762,25484820,425211,151616,


In [31]:
for index, row in nyt_avg_daily_cases.iterrows():
    date = index.date()
    nyt_avg_daily_cases.at[index, "date"] = date

In [32]:
nyt_avg_daily_cases.reset_index(drop=True, inplace=True)

In [33]:
nyt_avg_daily_cases.head(10)

,fips,cases,deaths,daily_new_cases,date
0,53,1,0,1,2020-01-21
1,53,1,0,0,2020-01-22
2,53,1,0,0,2020-01-23
3,70,2,0,1,2020-01-24
4,76,3,0,1,2020-01-25
5,80,5,0,2,2020-01-26
6,80,5,0,0,2020-01-27
7,80,5,0,0,2020-01-28
8,80,5,0,0,2020-01-29
9,80,6,0,1,2020-01-30


### Set Up NYT Totals Master DF

In [34]:
nyt_master = pd.merge(nytimes_nih_covid_df, state_pop_df, how ="inner", on="state")
nyt_master.head()

,date,state,fips,cases,deaths,population
0,2020-01-21,Washington,53,1,0,7614893
1,2020-01-22,Washington,53,1,0,7614893
2,2020-01-23,Washington,53,1,0,7614893
3,2020-01-24,Washington,53,1,0,7614893
4,2020-01-25,Washington,53,1,0,7614893


In [35]:
nyt_master["est_percent_infected"] = ""
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,
1,2020-01-22,Washington,53,1,0,7614893,
2,2020-01-23,Washington,53,1,0,7614893,
3,2020-01-24,Washington,53,1,0,7614893,
4,2020-01-25,Washington,53,1,0,7614893,


In [36]:
# Calcuation for est percent infected by state
for index, row in nyt_master.iterrows():
    population = int(row["population"])
    deaths = row["deaths"]
    infected = row["cases"]
    
    # take out the number who have passed away to reset estimated population
    est_population_new = population - deaths 
    
    # estimate the percent infected
    if infected == 0:
        percent_infected = 0
    else: 
        percent_infected = round((infected / est_population_new) * 100, 2)
    
    # insert into percent_infected column
    nyt_master.at[index, "est_percent_infected"] = percent_infected

In [37]:
nyt_master.head()

,date,state,fips,cases,deaths,population,est_percent_infected
0,2020-01-21,Washington,53,1,0,7614893,0
1,2020-01-22,Washington,53,1,0,7614893,0
2,2020-01-23,Washington,53,1,0,7614893,0
3,2020-01-24,Washington,53,1,0,7614893,0
4,2020-01-25,Washington,53,1,0,7614893,0


## JSON Exports

In [38]:
# Master DF to JSON
state_master_df.to_json("../data/state-master-data.json", orient="records")

In [39]:
# NYT Master to JSON 
nyt_master.to_json("../data/nyt-master.json", orient="records")

In [40]:
nyt_avg_daily_cases.to_json("../data/daily_new_cases.json", orient="records")

## Amazon RDS Updates

In [41]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [43]:
# 50 States & D.C. Data
state_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [44]:
# Daily Cases Data
nyt_avg_daily_cases.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)

In [45]:
# Raw NTY Data
nytimes_nih_covid_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)